In [86]:
from pyspark.sql import SparkSession
from datetime import date
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import udf, pandas_udf, PandasUDFType
spark = SparkSession.builder.getOrCreate()

In [87]:
df = spark.read.json('data100000.json')

In [88]:
df.show()
df.count()

+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|               track|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|[5ff24c931b913134...|       0|          3|             8|           15|             108|                null|        398|2020-12-30T15:00:00Z|243a0165719d0ba39...|        1|        |
|[5eebe4a41b913135...|       4|         22|           147|           24|             138|                null|        306|2020-01-16T18:00:00Z|930684efa2bc1e000...|        1|   28005|
|[5eec9c7e61804667...|       0|         11|            55|           14|        

100000

In [89]:
df = df.drop('_id','idplug_base','idunplug_base','track', 'zip_code')
df.show()
# to fit df in page and see more clearly the results

+--------+--------------+----------------+-----------+--------------------+--------------------+---------+
|ageRange|idplug_station|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|
+--------+--------------+----------------+-----------+--------------------+--------------------+---------+
|       0|             8|             108|        398|2020-12-30T15:00:00Z|243a0165719d0ba39...|        1|
|       4|           147|             138|        306|2020-01-16T18:00:00Z|930684efa2bc1e000...|        1|
|       0|            55|             177|       1285|2020-06-07T19:00:00Z|98c612931b2b1aaa4...|        1|
|       0|           149|              37|       1038|2020-08-30T14:00:00Z|c4fc91c7b4c2f0083...|        1|
|       0|            95|             149|        777|{"$date":"2018-09...|017be7543ec9e86a4...|        1|
|       4|             4|             130|        447|2020-09-28T17:00:00Z|c08d5c958726d256f...|        1|
|       0|            12|            

In [90]:
def get_weekday(hourtime):
    """
    Devuelve el día de la semana a partir de un iterador que contiene
    una fecha con un string
    0:Lunes,...,6:Domingo
    """

    try:
        year = int(hourtime[:4])
        month = int(hourtime[5:7])
        day = int(hourtime[8:10])

        return date(year, month, day).weekday()
    except BaseException:
        year = int(hourtime[10:14])
        month = int(hourtime[15:17])
        day = int(hourtime[18:20])

        return date(year, month, day).weekday()

day_func = udf(lambda x: get_weekday(x), IntegerType())
df.withColumn('get_weekday', day_func(df['unplug_hourTime'])).show()

+--------+--------------+----------------+-----------+--------------------+--------------------+---------+-----------+
|ageRange|idplug_station|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|get_weekday|
+--------+--------------+----------------+-----------+--------------------+--------------------+---------+-----------+
|       0|             8|             108|        398|2020-12-30T15:00:00Z|243a0165719d0ba39...|        1|          2|
|       4|           147|             138|        306|2020-01-16T18:00:00Z|930684efa2bc1e000...|        1|          3|
|       0|            55|             177|       1285|2020-06-07T19:00:00Z|98c612931b2b1aaa4...|        1|          6|
|       0|           149|              37|       1038|2020-08-30T14:00:00Z|c4fc91c7b4c2f0083...|        1|          6|
|       0|            95|             149|        777|{"$date":"2018-09...|017be7543ec9e86a4...|        1|          3|
|       4|             4|             130|      